In [51]:
import torch
from tqdm import tqdm
from model import seedformer_dim128
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d.loss import chamfer_distance
import numpy as np
import gc
#torch.cuda.empty_cache()
#gc.collect()
from torch.optim.lr_scheduler import StepLR
import utils.utils as utils

### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [2]:
class RacingDataset(Dataset):
    def __init__(self,root_dir,target_points=4731):
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()
        self.target_points = target_points

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)
        #padded_points = self.pad_point_cloud(pcd.points)

        return points,pcd_path

    #def pad_point_cloud(self, points):
    #    num_points = len(points)
    #    if num_points < self.target_points:
    #        # Pad the point cloud with zeros to match the target size
    #        padded_points = torch.cat([torch.tensor(points), torch.zeros(self.target_points - num_points, 3)])
    #    else:
    #        padded_points = torch.tensor(points)
    #    return padded_points
    
    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=0:
                filtered_list.append(filename)
        return filtered_list
   


In [3]:
#pcd_pad=o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
#pcd_pad_tens=torch.tensor(pcd_pad.points,dtype=torch.float32)
#print(pcd_pad_tens[0])

In [4]:
def collate_fn(batch):
    # Define the target number of points for padding
    target_num_points = 4731
    
    # Pad each point cloud to have target_num_points points
    padded_points_batch = []
    for points, _ in batch:
        num_points_to_pad = target_num_points - points.size(0)
        if num_points_to_pad > 0:
            pad = torch.zeros(num_points_to_pad, 3, dtype=torch.float32)
            padded_points = torch.cat((points, pad), dim=0)
        else:
            padded_points = points[:target_num_points]  # Trim to target_num_points if exceeds
        padded_points_batch.append(padded_points)
    
    padded_points = pad_sequence(padded_points_batch, batch_first=True, padding_value=0)
    
    paths = [item[1] for item in batch]
    
    return padded_points, paths
# Load the largest point cloud
pcd_pad = o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)

# Create the dataset and dataloader
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real", target_points=len(pcd_pad_tens))
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=8, collate_fn=collate_fn)

print(len(dataloader))
print(dataloader.dataset)

/tmp/ipykernel_5334/800869091.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)


750


In [5]:
#from numba import cuda
# 
#cuda.select_device(0) # choosing second GPU 
#cuda.close()

In [6]:
torch.manual_seed(42)
#del model
model=seedformer_dim128(up_factors=[1, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
epochs=100
optimizer=torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # Reduce LR by a factor of 0.1 every 30 epochs

for epoch in range(epochs):
    running_loss=0
    # Wrap the DataLoader with tqdm to track progress
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)  # Move data to GPU if available

            optimizer.zero_grad()
            outputs = model(inputs)
            losses = []

            for input_pc, output_pc in zip(inputs, outputs):
                # Calculate Chamfer Distance loss using pytorch3d.loss.chamfer_distance
                loss, _ = chamfer_distance(input_pc.unsqueeze(0), output_pc.unsqueeze(0))
                losses.append(loss)

            loss = torch.mean(torch.stack(losses))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))  # Update tqdm progress bar with the current loss
        scheduler.step()  # Step the learning rate scheduler

    print(f'Epoch {epoch+1} Loss: {running_loss / len(dataloader)}')  # Log the epoch loss

print('Finished Training')


cuda


Epoch 1/100: 100%|██████████| 750/750 [07:08<00:00,  1.75batch/s, loss=0.451]

Epoch 1 Loss: 0.45127651827860005



Epoch 2/100: 100%|██████████| 750/750 [08:05<00:00,  1.54batch/s, loss=0.00237]

Epoch 2 Loss: 0.002373430300639787


...

Epoch 64 Loss: 0.0001997098300714266



Epoch 65/100: 100%|██████████| 750/750 [06:37<00:00,  1.89batch/s, loss=0.0002]  

Epoch 65 Loss: 0.00019968022251424725



Epoch 66/100:  49%|████▉     | 366/750 [03:14<03:24,  1.88batch/s, loss=0.000237]


KeyboardInterrupt: 

In [7]:
#model=None

In [25]:
def apply_and_save_res(dataset,dataloader,model,savedir):
    model.eval()
    # Apply the model and visualize differences
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    for dataset, dataloader,  in [(dataset, dataloader)]:
        for i, data in enumerate(tqdm(dataloader, unit='point cloud')):
            #if i >= num_samples:
            #    break
            
            inputs, paths = data
            #print(paths)
            inputs = inputs.to(device)
    
            with torch.no_grad():
                outputs = model(inputs)
    
            outputs_cpu = outputs.cpu().numpy()

        #for path, output in zip(paths, outputs_cpu):
            filename = os.path.basename(paths[0])
            output_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(outputs_cpu[0]))
            f_name = os.path.join(savedir, filename)
            o3d.io.write_point_cloud(f_name, output_pcd)


## Save The Data


In [27]:
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)

# Load real dataset
real_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
real_dataloader = DataLoader(real_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)
apply_and_save_res(dataset=real_dataset,dataloader=real_dataloader,model=model,savedir="/home/omar/TUM/Data/reconstructed_cropped/real")

100%|██████████| 5993/5993 [03:44<00:00, 26.69point cloud/s]


## "Stitch" the data

In [104]:
def crop_invert_stitch(original_pcd,bbox_coords):
    orientation_angle = float(bbox_coords[-1])
    # print(orientation_angle)
    h, w, l = bbox_coords[8:11]
    rotation_mat_z = np.array([
        [np.cos(orientation_angle), -np.sin(orientation_angle), 0.0],
        [np.sin(orientation_angle), np.cos(orientation_angle), 0.0],
        [0.0, 0.0, 1.0]
    ])
    center = bbox_coords[11:14]
    bbox = o3d.geometry.OrientedBoundingBox(center=center, R=rotation_mat_z, extent=[l, w, h])
   # orignal_crop_invert =o3d.geometry.PointCloud.crop(original_pcd,bbox)
    inliers_indices = bbox.get_point_indices_within_bounding_box(original_pcd.points)
        
    inliers_pcd = original_pcd.select_by_index(inliers_indices, invert=False) # select inside points = cropped 
    outliers_pcd = original_pcd.select_by_index(inliers_indices, invert=True) #select outside points
    
    return outliers_pcd

In [ ]:
#source_label_path = "/home/omar/TUM/Masterarbeit/Data/m1695833/Sim2RealDistributionAlignedDataset/real/data/label/"
pcl=o3d.io.read_point_cloud("/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/pcl/018840.pcd")
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,1.56 ,6.55 ,1.01 ,-0.0]
pcd_cut=crop_invert_stitch(pcl,box)
utils.visualize(pcl=pcd_cut)

In [7]:
# Load simulation dataset
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)

# Load real dataset
real_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
real_dataloader = DataLoader(real_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=collate_fn)

# Select a subset of samples
num_samples = 3

# Apply the model and visualize differences
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

for dataset, dataloader, name in [(simulation_dataset, simulation_dataloader, "Simulation"), 
                                  (real_dataset, real_dataloader, "Real")]:
    for i, data in enumerate(tqdm(dataloader, desc=f'Processing {name} dataset', unit='point cloud')):
        if i >= num_samples:
            break
        
        inputs, paths = data
        #print(paths)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)

        original_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(inputs.squeeze().cpu().numpy()))
        original_pc.paint_uniform_color([1, 0, 0])  # Paint original point cloud in red

        reconstructed_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(outputs.squeeze().cpu().numpy()))
        reconstructed_pc.paint_uniform_color([0, 1, 0])  # Paint reconstructed point cloud in green

        o3d.visualization.draw_geometries([original_pc, reconstructed_pc], window_name=f"{name} - Sample {i+1}")
#model=None

Processing Simulation dataset:   0%|          | 0/5997 [00:00<?, ?point cloud/s]

['/home/omar/TUM/Data/cropped/sim/015580.pcd']


Processing Simulation dataset:   0%|          | 1/5997 [00:03<6:33:26,  3.94s/point cloud]

['/home/omar/TUM/Data/cropped/sim/015340.pcd']


Processing Simulation dataset:   0%|          | 2/5997 [00:05<4:14:53,  2.55s/point cloud]

...

Processing Real dataset:   0%|          | 1/5993 [00:01<1:40:04,  1.00s/point cloud]

['/home/omar/TUM/Data/cropped/real/022225.pcd']


Processing Real dataset:   0%|          | 2/5993 [00:01<1:26:18,  1.16point cloud/s]

['/home/omar/TUM/Data/cropped/real/006585.pcd']


Processing Real dataset:   0%|          | 3/5993 [00:02<1:39:06,  1.01point cloud/s]


In [ ]:
# Notes#
'''
input threshold adjust - 
'''